In [1]:
import pandas as pd
import os
import torch
#import torchaudio
#import requests
#import matplotlib
#import matplotlib.pyplot as plt
#import IPython
#import gc
import numpy as np

from os.path import isfile, join
from scipy.io.wavfile import write, read
from scipy.stats import kurtosis, skew
#from datasets import load_dataset
#from tqdm import tqdm
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model, Wav2Vec2ForPreTraining, Wav2Vec2ForXVector
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices

In [2]:
#home_path = '/home/hpc/iwi5/iwi5055h/'
home_path = ''
aibo_path = home_path + "data/AIBO/"
save_path = aibo_path + "wav2vec/"

In [3]:
#model_mode = 'base'
model_mode = 'xlsr'
#model_mode = 'xlsr-de'
#model_mode = 'xvec'
# 'xlsr' | 'base' | 'xvec'

In [4]:
def Wav2Vec_Models(file, model_name='xvec', plot=False, cuda=False):

    try:
        fs, audio=read(file)   
        audio=(audio - np.mean(audio))/max(audio)
       
        device='cpu'
        if cuda:
            device='cuda'
       
        #sampling_rate = fs
       
        if model_name=='xlsr':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
            model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
        elif model_name=='xlsr-de':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53-german")
            model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53-german").to(device)
        elif model_name=='base':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")   
            model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base-960h").to(device)
        elif model_name=='xvec':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("anton-l/wav2vec2-base-superb-sv")        
            #feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-er")        
            model = Wav2Vec2ForXVector.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
   
        # audio file is decoded on the fly
        input_values = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True).to(device).input_values
        input_values_all = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True)

        # compute masked indices
        batch_size, raw_sequence_length = input_values.shape
        sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length)
        mask_time_indices = _compute_mask_indices((batch_size, sequence_length), mask_prob=0.2, mask_length=2)
        mask_time_indices = torch.tensor(mask_time_indices, device=input_values.device, dtype=torch.long)

        with torch.inference_mode():

            tmp = model(**input_values_all.to(device))
            #tmp_proj=model_proj(input_values.to(device), mask_time_indices=mask_time_indices.to(device) )
            if model_name=='xvec':
                last_hidden_states = tmp.embeddings.cpu()
                #last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                features = tmp.logits.cpu()
                #features = np.hstack((features,features_std,features_skew,features_kurt,features_min,features_max))

                #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

            else:
                last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())
                last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())
                last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)
                last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)
                last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())
                last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())

                last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())
                features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())
                features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)
                features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)
                features_min=np.array(torch.min(tmp.extract_features, dim=1)[0].cpu())
                features_max=np.array(torch.max(tmp.extract_features, dim=1)[0].cpu())

                features = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

                #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

                #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

        return features[0], last_hidden_states[0]
    except:
        print('sequence not long enough')


### Testing a file

In [7]:
test_file = 'data/AIBO/wav/Mont_01_053_00.wav'

In [9]:
model_name = 'xlsr'
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [10]:
fs, audio=read(test_file)   
audio=(audio-np.mean(audio))/max(audio)
sampling_rate = fs

if model_name=='xlsr':
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
    model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
    #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
#elif model_name=='base':
 #   feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")   
  #  model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
   # #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base-960h").to(device)
#elif model_name=='xvec':
 #   feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("anton-l/wav2vec2-base-superb-sv")        
  #  model = Wav2Vec2ForXVector.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
    #feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-er")  # size mistmatch        
    #model = Wav2Vec2ForXVector.from_pretrained("superb/wav2vec2-base-superb-er").to(device)
    #model_proj = Wav2Vec2ForPreTraining.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.codevectors', 'project_q.bias', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# audio file is decoded on the fly
input_values = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True).to(device).input_values
#input_values_all = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True)

# compute masked indices
#batch_size, raw_sequence_length = input_values.shape
#sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length)
#mask_time_indices = _compute_mask_indices((batch_size, sequence_length), mask_prob=0.2, mask_length=2)
#mask_time_indices = torch.tensor(mask_time_indices, device=input_values.device, dtype=torch.long)

#with torch.inference_mode():
           
 #   tmp = model(**input_values_all.to(device))
    #tmp_proj=model_proj(input_values.to(device), mask_time_indices=mask_time_indices.to(device) )
  #  if model_name=='xvec':
   #     last_hidden_states = tmp.embeddings.cpu()
        #last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

    #    features = tmp.logits.cpu()
        #features = np.hstack((features,features_std,features_skew,features_kurt,features_min,features_max))

        #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

        #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))
'''
    else:
        last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())
        last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())
        last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)
        last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)
        last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())
        last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())

        last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

        features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())
        features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())
        features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)
        features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)
        features_min=np.array(torch.min(tmp.extract_features, dim=1)[0].cpu())
        features_max=np.array(torch.max(tmp.extract_features, dim=1)[0].cpu())

        features = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

        #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
        #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
        #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
        #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

        #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
        #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
        #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
        #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

print(features[0], last_hidden_states[0])
'''

'\n    else:\n        last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())\n        last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())\n        last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)\n        last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)\n        last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())\n        last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())\n\n        last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))\n\n        features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())\n        features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())\n        features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)\n        features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)\n        features_min=np.array(tor

## Creating .csv

In [87]:
#text_file = open('data/AIBO/labels/chunk_labels_5cl_corpus.txt')
text_file = open(home_path + 'data/AIBO/labels/chunk_labels_5cl_corpus.txt')
file_content = text_file.read()
#print(file_content)
text_file.close()

In [88]:
##data_ = text_file.decode("utf-8").split("\r\n")
data = file_content
data = data.split("\n")
for i in range(len(data)):
      data[i] = data[i].replace('_', ' ').split(" ")
#print(data)

In [89]:
labels_df = pd.DataFrame(data, columns=["source", "speaker", "n_rec", "num", "label", "percentage"], index=None)
labels_df['filename'] = labels_df.source + "_" + labels_df.speaker + "_" + labels_df.n_rec + "_" + labels_df.num
file_path = aibo_path + "wav/"
labels_df['path'] = file_path + labels_df['filename'] + ".wav"

In [90]:
labels_df

,source,speaker,n_rec,num,label,percentage,filename,path
0,Mont,01,000,00,N,1,Mont_01_000_00,data/AIBO/wav/Mont_01_000_00.wav
1,Mont,01,001,00,N,1,Mont_01_001_00,data/AIBO/wav/Mont_01_001_00.wav
2,Mont,01,001,01,N,1,Mont_01_001_01,data/AIBO/wav/Mont_01_001_01.wav
3,Mont,01,004,00,N,0.9,Mont_01_004_00,data/AIBO/wav/Mont_01_004_00.wav
4,Mont,01,005,00,N,1,Mont_01_005_00,data/AIBO/wav/Mont_01_005_00.wav
...,...,...,...,...,...,...,...,...
18212,Ohm,32,318,00,E,0.4,Ohm_32_318_00,data/AIBO/wav/Ohm_32_318_00.wav
18213,Ohm,32,319,00,E,0.6,Ohm_32_319_00,data/AIBO/wav/Ohm_32_319_00.wav
18214,Ohm,32,320,00,N,0.9,Ohm_32_320_00,data/AIBO/wav/Ohm_32_320_00.wav
18215,Ohm,32,321,00,P,0.7,Ohm_32_321_00,data/AIBO/wav/Ohm_32_321_00.wav


In [91]:
len(labels_df)

18217

In [92]:
#testdf = labels_df

In [47]:
#testdf['spsource'] = testdf['source'] + testdf['speaker']

In [48]:
#testdf.spsource.value_counts()

In [49]:
#test_file = "data/AIBO/wav/Mont_01_007_00.wav"

In [50]:
#fs, audio = read(test_file)

In [51]:
#fs

In [93]:
labels_df["speaker"] = pd.to_numeric(labels_df["speaker"])

In [53]:
#labels_df.speaker.value_counts()

In [54]:
##labels_df.loc[labels_df.source =='Ohm', 'speaker'] = labels_df.speaker + 25
# speaker 01 from Ohm or Mont should be treated equally

In [94]:
labels_dict = {'A': 0, 'E': 1, 'N': 2, 'P': 3, 'R': 4}
labels_df['lb'] = labels_df['label'].map(labels_dict)

In [95]:
labels_df

,source,speaker,n_rec,num,label,percentage,filename,path,lb
0,Mont,1.0,000,00,N,1,Mont_01_000_00,data/AIBO/wav/Mont_01_000_00.wav,2.0
1,Mont,1.0,001,00,N,1,Mont_01_001_00,data/AIBO/wav/Mont_01_001_00.wav,2.0
2,Mont,1.0,001,01,N,1,Mont_01_001_01,data/AIBO/wav/Mont_01_001_01.wav,2.0
3,Mont,1.0,004,00,N,0.9,Mont_01_004_00,data/AIBO/wav/Mont_01_004_00.wav,2.0
4,Mont,1.0,005,00,N,1,Mont_01_005_00,data/AIBO/wav/Mont_01_005_00.wav,2.0
...,...,...,...,...,...,...,...,...,...
18212,Ohm,32.0,318,00,E,0.4,Ohm_32_318_00,data/AIBO/wav/Ohm_32_318_00.wav,1.0
18213,Ohm,32.0,319,00,E,0.6,Ohm_32_319_00,data/AIBO/wav/Ohm_32_319_00.wav,1.0
18214,Ohm,32.0,320,00,N,0.9,Ohm_32_320_00,data/AIBO/wav/Ohm_32_320_00.wav,2.0
18215,Ohm,32.0,321,00,P,0.7,Ohm_32_321_00,data/AIBO/wav/Ohm_32_321_00.wav,3.0


In [96]:
len(labels_df)

18217

### Splitting dataset

In [97]:
labels_df.source.value_counts()

Ohm     9959
Mont    8257
           1
Name: source, dtype: int64

In [98]:
def split_indices(last=25, split_ratio=0.7, seed=13):
    np.random.seed(seed)
    train_size = int(last * split_ratio)
    val_size = int((1 - split_ratio)/2 * last)
    test_size = last - train_size - val_size
    
    a = np.arange(1, last+1, 1)
    b = np.random.permutation(a)
    chunk_size = [train_size, val_size, test_size]
    np.cumsum(chunk_size)
    c = np.split(b, np.cumsum(chunk_size))
    train, val, test = c[0], c[1], c[2]
    if len(c[3]) > 0:
        print("some samples left unused while splitting")
    return train, val, test

In [99]:
last = int(max(dict((labels_df.speaker).value_counts()).keys()))
last

32

In [100]:
train_indices, val_indices, test_indices = split_indices(last, 0.7, 6)
split_indices(last, 0.7, 6)

(array([ 8, 29, 22,  1, 24, 31, 30, 25,  7,  6, 18, 20,  3,  9, 19,  5, 15,
        13, 23, 26, 27, 12]),
 array([ 2, 17, 28, 16]),
 array([14, 32, 21,  4, 10, 11]))

In [101]:
labels_df['split']=labels_df['speaker'].apply(lambda x: 'train' if x in train_indices else ('test' if x in test_indices else 'val'))

In [102]:
labels_df.split.value_counts()

train    12284
test      3671
val       2262
Name: split, dtype: int64

In [103]:
labels_df = labels_df.dropna()

In [104]:
len(labels_df)

18216

In [105]:
labels_df.label.value_counts()

N    10967
E     3601
A     1492
R     1267
P      889
Name: label, dtype: int64

In [106]:
labels_df.to_csv(aibo_path + 'labels_df.csv')

## Extracting .npy samplewise

In [5]:
labels_df = pd.read_csv(aibo_path + 'labels_df.csv', header=0, index_col=0)

In [6]:
len(labels_df)

18216

In [7]:
labels_df

,source,speaker,n_rec,num,label,percentage,filename,path,lb,split
0,Mont,1.0,0,0,N,1.0,Mont_01_000_00,data/AIBO/wav/Mont_01_000_00.wav,2.0,train
1,Mont,1.0,1,0,N,1.0,Mont_01_001_00,data/AIBO/wav/Mont_01_001_00.wav,2.0,train
2,Mont,1.0,1,1,N,1.0,Mont_01_001_01,data/AIBO/wav/Mont_01_001_01.wav,2.0,train
3,Mont,1.0,4,0,N,0.9,Mont_01_004_00,data/AIBO/wav/Mont_01_004_00.wav,2.0,train
4,Mont,1.0,5,0,N,1.0,Mont_01_005_00,data/AIBO/wav/Mont_01_005_00.wav,2.0,train
...,...,...,...,...,...,...,...,...,...,...
18211,Ohm,32.0,317,1,N,0.8,Ohm_32_317_01,data/AIBO/wav/Ohm_32_317_01.wav,2.0,test
18212,Ohm,32.0,318,0,E,0.4,Ohm_32_318_00,data/AIBO/wav/Ohm_32_318_00.wav,1.0,test
18213,Ohm,32.0,319,0,E,0.6,Ohm_32_319_00,data/AIBO/wav/Ohm_32_319_00.wav,1.0,test
18214,Ohm,32.0,320,0,N,0.9,Ohm_32_320_00,data/AIBO/wav/Ohm_32_320_00.wav,2.0,test


In [8]:
def w2v_path(file, split):    
    return save_path + model_mode + "_extract_features/" + split + '-' + file + '.npy'

In [9]:
na_files = []
problems = []

In [10]:
for k,v in labels_df.iterrows():
    try:
        if not isfile(v['path']):
            na_files.append(v['path'])
            print("wav for {} not available". format(v['filename']))
        elif isfile(w2v_path(v['filename'], v['split'])):
            pass
            #print("w2v features found for {}".format(v['filename']))
        else:
            #ft, hs = Wav2Vec_Model(v['path'])
            ft, hs = Wav2Vec_Models(v['path'], model_name=model_mode, cuda=True)
            #np.save(save_path + "extract_features/" + v['split'] + '-' + v['filename'] + '.npy', ft)
            np.save(save_path + model_mode + "_extract_features/" + v['split'] + '-' + v['filename'] + '.npy', ft)
            np.save(save_path + model_mode + "_hidden_states/" + v['split'] + '-' + v['filename'] + '.npy', hs)
            print("saved for {}".format(v['filename']))
    except:
        problems.append(v['filename'])
        print("wrong")

In [11]:
na_files

[]

In [12]:
problems

[]

## Gathering .npy arrays into train/val/test with features, hidden states and layers

In [13]:
aibo_ft_train = []
aibo_ft_val = []
aibo_ft_test = []

In [14]:
aibo_hs_train = []
aibo_hs_val = []
aibo_hs_test = []

In [15]:
aibo_lb_train = []
aibo_lb_val = []
aibo_lb_test = []

In [16]:
for file in os.listdir(save_path + model_mode + "_extract_features/"):
    if file.startswith("train-"):
        #filename = file.split("-")[1]
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        #if len(ft)> 3000:
        aibo_ft_train.append(ft)
    elif file.startswith("val-"):
        #filename = file.split("-")[1]
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        aibo_ft_val.append(ft)
    else:
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        aibo_ft_test.append(ft)        
np.save(save_path + model_mode + "_aibo_ft_train.npy", aibo_ft_train)
np.save(save_path + model_mode + "_aibo_ft_val.npy", aibo_ft_val)
np.save(save_path + model_mode + "_aibo_ft_test.npy", aibo_ft_test)

In [17]:
for file in os.listdir(save_path + model_mode + "_hidden_states/"):
    if file.startswith("train-"):
        #filename = file.split("-")[1]
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        aibo_hs_train.append(hs)
    elif file.startswith("val-"):
        #filename = file.split("-")[1]
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        aibo_hs_val.append(hs)
    else:
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        aibo_hs_test.append(hs)        
np.save(save_path + model_mode + "_aibo_hs_train.npy", aibo_hs_train)
np.save(save_path + model_mode + "_aibo_hs_val.npy", aibo_hs_val)
np.save(save_path + model_mode + "_aibo_hs_test.npy", aibo_hs_test)

In [18]:
labels_df.dtypes

source         object
speaker       float64
n_rec           int64
num             int64
label          object
percentage    float64
filename       object
path           object
lb            float64
split          object
dtype: object

In [19]:
for file in os.listdir(save_path + model_mode + "_extract_features/"):
    label = labels_df.loc[labels_df['filename'] == file.split("-")[1].split(".")[0], 'lb']
    #print(type(label))
    #print(type(int(float(label.values))))
    #print(type(str(label)))
    #lb = int(float(label.replace('\n', ' ').split(' ')[4]))
    lb = str(label)
    #lb = pd.to_string(label, index=False)
    #lb = label
    lb = int(float(lb.replace('\n', ' ').split(' ')[4]))
    if file.startswith("train-"):
        aibo_lb_train.append(lb)
    elif file.startswith("val-"):
        aibo_lb_val.append(lb)
    else:
        aibo_lb_test.append(lb)
np.save(save_path + model_mode + "_aibo_lb_train.npy", aibo_lb_train)
np.save(save_path + model_mode + "_aibo_lb_val.npy", aibo_lb_val)
np.save(save_path + model_mode + "_aibo_lb_test.npy", aibo_lb_test)

In [20]:
#len(os.listdir(save_path + model_mode + "_extract_features/"))

In [21]:
len(aibo_lb_val)

2261

In [22]:
len(aibo_lb_train)

12284

In [23]:
len(aibo_lb_val)

2261

In [ ]:
#len(aibo_lb_test)

In [ ]:
#len(aibo_hs_test)

In [ ]:
#aibo_lb_train[634]

In [26]:
#type(aibo_lb_test[0].replace)

In [27]:
#len(aibo_lb_val)

84

In [118]:
#str(aibo_lb_val[0]).replace('\n', ' ').split(' ')

['2']

In [137]:
#for i in range(len(aibo_lb_val)):
 #   a = str(aibo_lb_val[i]).replace('\n', ' ').split(' ')[4]
  #  aibo_lb_val[i] = int(float(a))

In [138]:
#type(int(float(aibo_lb_test[0])))

int

In [31]:
#len(aibo_lb_train[2].split(' '))

8

In [40]:
#int(float(aibo_lb_train[2].replace('\n', ' ').split(' ')[4]))

1

In [39]:
#type(int(float(aibo_lb_train[2].replace('\n', ' ').split(' ')[4])))

int

In [ ]:
#np.unique(labels)

In [17]:
#len(os.listdir('data/AIBO/wav/'))

3547

In [21]:
#len(na_files)

14874